In [6]:
from skimage import feature
from skimage.color import rgb2gray
from skimage.transform import resize
from skimage.morphology import erosion, dilation, rectangle, square
from skimage.measure import find_contours

from scipy import ndimage as ndi
from scipy.misc import imsave

%matplotlib inline

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import cv2

from collections import defaultdict
from copy import deepcopy

#import pyscreenshot as pysc
from PIL import ImageGrab
# from PIL import Image

import random
import math
from time import sleep

In [7]:
colormode = 'L'
mystery = ndi.imread('grid_square_mystery.png', mode=colormode)
empty = ndi.imread('grid_square_empty.png',  mode=colormode)
grid_one = ndi.imread('grid_one.png', mode=colormode)
grid_two = ndi.imread('grid_two.png', mode=colormode)
grid_three = ndi.imread('grid_three.png', mode=colormode)
grid_four = ndi.imread('grid_four.png', mode=colormode)
grid_five = ndi.imread('grid_five.png', mode=colormode)
grid_six = ndi.imread('grid_six.png', mode=colormode)
grid_seven = ndi.imread('grid_seven.png', mode=colormode)
#minesweep = ndi.imread('My_shot.png', mode='L')
flag = ndi.imread('flag.png', mode=colormode)
smiley_face = ndi.imread('smiley_face.png', mode=colormode)
frowny_face = ndi.imread('frowny_face.png', mode=colormode)
glasses_face = ndi.imread('glasses_face.png', mode=colormode)
difficulty_level = 1

In [8]:
if difficulty_level == 1:
    x_left = -275
    x_right = 335
    y_top = -85
    y_bot = 690
    grid_rows = 9
    grid_cols = 9
    mine_total = 10
elif difficulty_level == 2:
    x_left = -490
    x_right = 545
    y_top = -85
    y_bot = 1100
    grid_rows = 16
    grid_cols = 16
    mine_total = 40
elif difficulty_level == 3:
    x_left = -900
    x_right = 955
    y_top = -85
    y_bot = 1100
    grid_rows = 16
    grid_cols = 30
    mine_total = 99

In [9]:
def dist((x1,y1),(x2,y2)):
    return math.sqrt((x2-x1)**2 + (y2-y1)**2)

In [10]:
def remove_close_points(spots, similarity_matrix, threshold = 5):
    spotscopy = deepcopy(spots)
    for i in range(len(spotscopy)):
        for j in range(len(spotscopy)):
            a = spotscopy[i]
            b = spotscopy[j]
            if dist(a,b) < threshold and j > i:
                if similarity_matrix[a[0]][a[1]] > similarity_matrix[b[0]][b[1]] and b in spots:
                    spots.remove(b)
                elif similarity_matrix[a[0]][a[1]] <= similarity_matrix[b[0]][b[1]] and a in spots:
                    spots.remove(a)
    return spots

In [11]:
def find_candidate_locations(similarity_matrix):
    candidate_spots = []
    thing = np.where(similarity_matrix > .95)
#     for n in range(len(similarity_matrix)):
#         for m in range(len(similarity_matrix[n])):
#             if similarity_matrix[n][m] > .95:
#                 candidate_spots.append((n,m))
    return zip(thing[0], thing[1])
#     final_spots = remove_close_points(candidate_spots, similarity_matrix)
#     return final_spots

In [12]:
from Quartz.CoreGraphics import CGEventCreateMouseEvent
from Quartz.CoreGraphics import CGEventPost
from Quartz.CoreGraphics import kCGEventMouseMoved
from Quartz.CoreGraphics import kCGEventLeftMouseDown
from Quartz.CoreGraphics import kCGEventLeftMouseDown
from Quartz.CoreGraphics import kCGEventLeftMouseUp
from Quartz.CoreGraphics import kCGMouseButtonLeft
from Quartz.CoreGraphics import kCGHIDEventTap
from Quartz.CoreGraphics import kCGEventRightMouseDown
from Quartz.CoreGraphics import kCGEventRightMouseUp

def mouseEvent(type, posx, posy):
        theEvent = CGEventCreateMouseEvent(
                    None, 
                    type, 
                    (posx,posy), 
                    kCGMouseButtonLeft)
        CGEventPost(kCGHIDEventTap, theEvent)

def mousemove(posx,posy):
        mouseEvent(kCGEventMouseMoved, posx,posy);

def mouseclick(posx,posy):
        # uncomment this line if you want to force the mouse 
        # to MOVE to the click location first (I found it was not necessary).
        #mouseEvent(kCGEventMouseMoved, posx,posy);
        mouseEvent(kCGEventLeftMouseDown, posx,posy);
        mouseEvent(kCGEventLeftMouseUp, posx,posy);

def rightmouseclick(posx,posy):
        mouseEvent(kCGEventRightMouseDown, posx,posy);
        mouseEvent(kCGEventRightMouseUp, posx,posy);

In [13]:
#mouseclick(max_loc[0]/2 + 100, max_loc[1]/2 + 50)

In [14]:
#rightmouseclick(980,500)

In [15]:
def generate_grid(picture):
    #print 'GRID'
    #print grid
    temp_grid = deepcopy(grid)
    mystery_squares = np.where(temp_grid == -1)
#     print 'MYSTERY SQUARES'
    mystery_squares = zip(mystery_squares[0], mystery_squares[1])
    #print mystery_squares
    #print grid_coord_dict

    #gridss = cv2.matchTemplate(fs, picture, cv2.TM_CCOEFF_NORMED)
    ones = cv2.matchTemplate(picture, grid_one, cv2.TM_CCOEFF_NORMED)
    twos = cv2.matchTemplate(picture, grid_two, cv2.TM_CCOEFF_NORMED)
    threes = cv2.matchTemplate(picture, grid_three, cv2.TM_CCOEFF_NORMED)
    fours = cv2.matchTemplate(picture, grid_four, cv2.TM_CCOEFF_NORMED)
    fives = cv2.matchTemplate(picture, grid_five, cv2.TM_CCOEFF_NORMED)
    sixes = cv2.matchTemplate(picture, grid_six, cv2.TM_CCOEFF_NORMED)
    sevens = cv2.matchTemplate(picture, grid_seven, cv2.TM_CCOEFF_NORMED)
    empties = cv2.matchTemplate(picture, empty, cv2.TM_CCOEFF_NORMED)
    mysteries = cv2.matchTemplate(picture, mystery, cv2.TM_CCOEFF_NORMED)
    #flags = cv2.matchTemplate(picture, flag, cv2.TM_CCOEFF_NORMED)
    smileys = cv2.matchTemplate(picture, smiley_face, cv2.TM_CCOEFF_NORMED)
    frownys = cv2.matchTemplate(picture, frowny_face, cv2.TM_CCOEFF_NORMED)
    glassys = cv2.matchTemplate(picture, glasses_face, cv2.TM_CCOEFF_NORMED)

    adj_x = 4
    adj_y = 4
    smiley_max = cv2.minMaxLoc(smileys)[1]
    frowny_max = cv2.minMaxLoc(frownys)[1]
    glasses_max = cv2.minMaxLoc(glassys)[1]
    winorlose = np.argmax([smiley_max,frowny_max,glasses_max])
    if winorlose > 0:
        return []
    #if gridss[max_loc[1] + y_top, max_loc[0] + x_left] < .5:
        #return []
        
    #print 'I finished making a grid'
    #there is a clear path to rid ourselves of this for loop. Implement later.
    #I am looking back and have no idea how to rid myself of this for loop.
    for coords in mystery_squares:
        loc = grid_coord_dict[coords]
        one_sim = ones[loc[0] + adj_x, loc[1] + adj_y]
        two_sim = twos[loc[0] + adj_x, loc[1] + adj_y]
        three_sim = threes[loc[0] + adj_x, loc[1] + adj_y]
        four_sim = fours[loc[0] + adj_x, loc[1] + adj_y]
        five_sim = fives[loc[0] + adj_x, loc[1] + adj_y]
        six_sim = sixes[loc[0] + adj_x, loc[1] + adj_y]
        seven_sim = sevens[loc[0] + adj_x, loc[1] + adj_y]
        empty_sim = empties[loc[0] + adj_x, loc[1] + adj_y]
        mystery_sim = mysteries[loc[0], loc[1]]
        #flag_sim = flags[loc[0], loc[1]]
        ind = np.argmax([empty_sim, one_sim, two_sim, three_sim, four_sim, five_sim, six_sim, seven_sim, mystery_sim])
        if ind == 8:
            idx = -1
        else:
            idx = ind
        #print my_grid
        #print coords
        temp_grid[coords[0], coords[1]] = idx
    #print 'finished making a grid!'
    return temp_grid

In [16]:
def finalize_grid_locations():
    fullscreen = ImageGrab.grab().convert(colormode)# X1,Y1,X2,Y2
    global fs
    fs = np.array(fullscreen)
    simil_mat = cv2.matchTemplate(fs, smiley_face, cv2.TM_CCOEFF_NORMED)
    thing = cv2.minMaxLoc(simil_mat)
    max_loc = thing[3]
    mouseclick(max_loc[0]/2 - 10, max_loc[1]/2)
    mines = ImageGrab.grab(bbox = (max_loc[0] + x_left, max_loc[1] + y_top, max_loc[0] + x_right, max_loc[1] + y_bot)).convert(colormode)  # X1,Y1,X2,Y2
    #mines.show()
    mines = np.array(mines)
    mysteries = cv2.matchTemplate(mines, mystery, cv2.TM_CCOEFF_NORMED)
    grid_locations = remove_close_points(find_candidate_locations(mysteries), mysteries)
    return max_loc, grid_locations

In [17]:
def click_on_square(coords):
    click_target_x = (max_loc[0] + x_left + grid_coord_dict[coords][1])/2 + 15
    click_target_y= (max_loc[1] + y_top + grid_coord_dict[coords][0])/2 + 15
    mouseclick(max_loc[0]/2 -10, max_loc[1]/2)
    mouseclick(click_target_x, click_target_y)

In [18]:
def right_click_on_square(coords):
#     if grid[coords[0],coords[1]] != -1:
#         print grid[coords[0],coords[1]]
    if coords in set_of_clicks:
        print 'I messed up'
    set_of_clicks.add(coords)
    grid[coords[0],coords[1]] = -3
    click_target_x = (max_loc[0] + x_left + grid_coord_dict[coords][1])/2 + 15
    click_target_y= (max_loc[1] + y_top + grid_coord_dict[coords][0])/2 + 15
    mouseclick(max_loc[0]/2 -10, max_loc[1]/2)
    rightmouseclick(click_target_x,click_target_y)
    #rightmouseclick(click_target_x,click_target_y)

In [19]:
def check_adjacent_list(coords_1, coords_list):
    for coord in coords_list:
        if check_adjacent(coords_1, coord):
            return True
    return False

In [20]:
def check_adjacent(coords_1, coords_2):
    x1 = coords_1[0]
    x2 = coords_2[0]
    y1 = coords_1[1]
    y2 = coords_2[1]
    if abs(x1 - x2) <= 2 and abs(y1 - y2) <= 2:
        adj_1 = get_adjacent(grid, coords_1)
        adj_2 = get_adjacent(grid, coords_2)
        num_coords_1 = np.where(adj_1.reshape((3,3)) > 0)
        num_coords_1 = zip(map(lambda x: x - 1 + x1, num_coords_1[0]), map(lambda y: y - 1 + y1, num_coords_1[1]))
        num_coords_2 = np.where(adj_2.reshape((3,3)) > 0)
        num_coords_2 = zip(map(lambda x: x - 1 + x2, num_coords_2[0]), map(lambda y: y - 1 + y2, num_coords_2[1]))
        if len(set.intersection(set(num_coords_1), set(num_coords_2))) > 0:
            return True
        else:
            return False
    else:
        return False

In [21]:
def get_border_mysteries():
    border_squares = []
    mystery_squares = np.where(grid == -1)
    mystery_squares = zip(mystery_squares[0], mystery_squares[1])
    big_list = []
    mines_left = mine_total - len(np.where(grid == -3)[0])
    
    for coords in mystery_squares:
        adjacents = get_adjacent(grid, coords)
         #print coords
        if sum(np.in1d(adjacents, np.arange(1,9))) > 0:
             border_squares.append(coords) 
    if mines_left < 10:
        print 'LESS THAN 10 MINES'
        return [border_squares]
    #print 'border_squares'
    #print border_squares 
    while len(border_squares) > 0:
        start_square = border_squares.pop(0)
        finding_stuff = True
        start_list = [start_square]
        while finding_stuff:
            i = 0
            finding_stuff = False
            while i < len(border_squares):
                sqr = border_squares[i]
                if check_adjacent_list(sqr, start_list):
                    start_list.append(border_squares.pop(i))
                    finding_stuff = True
                else:
                    i += 1
        big_list.append(start_list)
    return big_list

In [22]:
def get_adjacent(my_grid, coords):
    i = coords[0] + 1
    j = coords[1] + 1
    temp_grid = np.multiply(np.ones((my_grid.shape[0] + 2, my_grid.shape[1] + 2)),-10)
    #print temp_grid.shape
    temp_grid[1:my_grid.shape[0] + 1, 1:my_grid.shape[1] + 1] = deepcopy(my_grid)
    adj = np.concatenate((temp_grid[i-1][j-1:j+2],temp_grid[i][j-1:j+2],temp_grid[i+1][j-1:j+2]))
    return adj

In [23]:
import sys

In [24]:
sys.setrecursionlimit(1500)

In [25]:
def backtracker(lists, border_squares):
    list_of_lists = []
    found_a_branch = False
#     print 'first'
#     print len(lists)
    if len(lists) == 0:
        #print "I'm here"
        #-2 means open, -4 means mine.
        lis_one = np.array([-2])
        lis_two = np.array([-4])
        if check_valid(lis_one, border_squares):
                list_of_lists.append(lis_one)
        if check_valid(lis_two, border_squares):
                list_of_lists.append(lis_two)  
    else:
#         print '2nd'
#         print len(lists[0])
        for arr in lists:
            lis_one = np.append(arr,-2)
            lis_two = np.append(arr,-4)
            if check_valid(lis_one, border_squares):
                list_of_lists.append(lis_one)
            if check_valid(lis_two, border_squares):
                list_of_lists.append(lis_two) 
    #print found_a_branch
#     print 'lenlistone'
#     print len(lis_one)
#     print 'lenlisttwo'
#     print len(lis_two)
#     print 'lenbordersqrs'
#     print len(border_squares)
    if len(lis_one) == len(border_squares) or len(list_of_lists) ==0:
        return list_of_lists
    else:
        return backtracker(list_of_lists, border_squares)     

In [26]:
def check_valid(arr, border_squares):
    temp_grid = deepcopy(grid)
    is_valid = True
    #if len(arr) != 2:
    #print arr
    #already_checked = np.zeros(temp_grid.shape)
    
    mines_left = mine_total - len(np.where(temp_grid <= -3)[0])
#     print ''
#     print 'mines'
#     print mines_left
    for i in range(len(arr)):
        temp_grid[border_squares[i]] = arr[i]
    #for j in range(len(arr)):
    coords = border_squares[len(arr) - 1]
    adjs = get_adjacent(temp_grid, coords)
    num_locs = np.where(adjs.reshape((3,3)) > 0)
    num_coords = zip(map(lambda x: x - 1 + coords[0],num_locs[0]), map(lambda y: y - 1 + coords[1], num_locs[1]))
    for coords_2 in num_coords:
        adjs_2 = get_adjacent(temp_grid, coords_2)
        bomb_count = sum(adjs_2 == -3) + sum(adjs_2 == -4)
        hypo_bomb_count = sum(adjs_2 == -4)
        mystery_count = sum(adjs_2 == -1)
        if bomb_count > adjs_2[4]:
            #print 'bomb high'
            is_valid = False
            return is_valid
        if bomb_count < adjs_2[4] and mystery_count == 0:
            #print 'bomb low'
            is_valid = False
            return is_valid
        if hypo_bomb_count > mines_left:
            is_valid = False
            return is_valid
    return is_valid

In [27]:
def check_for_commonalities(mine_list, border_squares):
    #print mine_list
    mine_list_rot = zip(*mine_list)
    did_something = False
    prob_list = []
    #print mine_list_rot
    for i in range(len(mine_list_rot)):
        possibilities = list(mine_list_rot[i])
        #print type(possibilities)
        #print possibilities
        if len(set(possibilities)) == 1 and list(set(possibilities))[0] == -4:
            right_click_on_square(border_squares[i])
            did_something = True
        if len(set(possibilities)) == 1 and list(set(possibilities))[0] == -2:
            click_on_square(border_squares[i])
            did_something = True
#         else:
#             prob_list.append(sum(mine_list_rot[i] == -4)/float(len(mine_list_rot[i])))
            
    return did_something#, prob_list

In [28]:
def choose_a_rando(mine_list, border_squares):
    #print mine_list
    mine_list_rot = zip(*mine_list)
#     print 'mine_list_rot'
#     print len(mine_list_rot)
#     print 'border squares'
#     print len(border_squares)
    did_something = False
    prob_list = []
    mine_list_probs = []
    for possibilities in mine_list_rot:
        mine_list_probs.append(sum(np.array(possibilities) == -4))
    tempor_grid = np.zeros((grid_rows, grid_cols))
    
    for i in range(len(mine_list_probs)):
        prob = mine_list_probs[i]
        sqr = border_squares[i]
        tempor_grid[sqr] = prob
    print tempor_grid
#     square_to_click = np.argmin(mine_list_probs)
#     click_on_square(border_squares[square_to_click])

In [29]:
v = [1,2,3]

In [30]:
np.argmax(v)

2

In [31]:
def brute_force_algorithm():
    #border_squares = []
    border_squares = get_border_mysteries()
    #sleep(.5)    
#     print 'bordersquares'
#     print border_squares
    something_done = False
    thing = False
    #big_prob_list = []
    #print len(border_squares)
    if len(border_squares) > 0:
        for border_cluster in border_squares:
            mine_list = backtracker([], border_cluster)
            thing = check_for_commonalities(mine_list, border_cluster)
            if thing:
                something_done = True
        if something_done:
            print 'i did someting'
        else:
            print 'something was not done!'
            flat_border_squares = sum(border_squares, [])
            mine_list = backtracker([], flat_border_squares)
            choose_a_rando(mine_list, flat_border_squares)
    else:
        print 'first move only'
#         print grid
        mystery_squares = np.where(grid == -1)
        #print mystery_squares
        mystery_squares = zip(mystery_squares[0], mystery_squares[1])
        n = random.randint(0, len(mystery_squares) - 1)
        click_on_square(mystery_squares[n])
        something_done = True
    
    return True

In [32]:
def play_minesweeper():
#     subprocess.call(
#     ["/usr/bin/open", "-n", "-a", "/Applications/Minesweeper Deluxe.app"]
#     )
#     sleep(5)
    global set_of_clicks
    set_of_clicks = set()
    global max_loc
    #print max_loc
    global grid_locations
    max_loc, grid_locations = finalize_grid_locations()
    #print grid_locations
    #create dictionary between locations and coordinates
    
    global grid_loc_dict
    grid_loc_dict = dict()
    #print grid_locations
    for i in range(len(grid_locations)):
        tup = grid_locations[i]
        grid_loc_dict[tup] = (i/grid_cols, i % grid_cols)

    global grid_coord_dict
    grid_coord_dict = dict()
    for x in grid_loc_dict.items():
        grid_coord_dict[x[1]] = x[0]
    global grid
    grid = np.negative(np.ones((grid_rows,grid_cols)))
    
    while(len(grid) > 0):
        ble = brute_force_algorithm()
        sleep(.1)
        mines2 = ImageGrab.grab(bbox = (max_loc[0] + x_left, max_loc[1] + y_top, max_loc[0] + x_right, max_loc[1] + y_bot)).convert(colormode)  # X1,Y1,X2,Y2
        #mines2.show()
        mines2 = np.array(mines2)
        #global grid
        grid = generate_grid(mines2)
        #sleep(.5)
#         print ''
#         print ''
#         print grid

In [40]:
play_minesweeper()

first move only
i did someting
something was not done!
[[  0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   6.   6.   6.   0.   0.   0.   0.   0.]
 [  9.  12.   0.  12.  11.   0.   0.   0.   0.]
 [  0.   0.   0.   0.  11.   0.   0.   0.   0.]
 [  0.   0.   0.  18.  11.   0.   0.   0.   0.]
 [  0.   0.   0.   3.   0.   0.   0.   0.   0.]]
something was not done!
[[  0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   6.   6.   6.   0.   0.   0.   0.   0.]
 [  9.  12.   0.  12.  11.   0.   0.   0.   0.]
 [  0.   0.   0.   0.  11.   0.   0.   0.   0.]
 [  0.   0.   0.  18.  11.   0.   0.   0.   0.]
 [  0.   0.   0.   3.   0.   0.   0.   0.   0.]]
something was not done!
[[  0.   0.   0.   0.   0.   0.

ValueError: empty range for randrange() (0,0, 0)

In [144]:
max_loc, grid_locations = finalize_grid_locations()
    #print grid_locations
    #create dictionary between locations and coordinates
global grid_loc_dict
grid_loc_dict = dict()
for i in range(len(grid_locations)):
    tup = grid_locations[i]
    grid_loc_dict[tup] = (i/grid_rows, i % grid_rows)

global grid_coord_dict
grid_coord_dict = dict()
for x in grid_loc_dict.items():
    grid_coord_dict[x[1]] = x[0]
global grid
grid = np.negative(np.ones((grid_rows,grid_cols)))

In [ ]:
mines2 = ImageGrab.grab(bbox = (max_loc[0] + x_left, max_loc[1] + y_top, max_loc[0] + x_right, max_loc[1] + y_bot)).convert(colormode)  # X1,Y1,X2,Y2
mines2 = np.array(mines2)
grid = generate_grid(mines2)

In [ ]:
thing = get_border_mysteries(grid)

In [ ]:
len(thing[2])

In [ ]:
temp_grid = map(lambda x: map(lambda y: (y,0),x), grid)

In [ ]:
from copy import deepcopy

In [ ]:
def check_valid_old(arr, border_squares):
    temp_grid = deepcopy(grid)
    is_valid = True
    #already_checked = np.zeros(temp_grid.shape)
    for i in range(len(arr)):
        temp_grid[border_squares[i]] = arr[i]
    for j in range(len(arr)):
        coords = border_squares[i]
        adjs = get_adjacent(temp_grid, coords)
        num_locs = np.where(adjs.reshape((3,3)) > 0)
        num_locs = zip(map(lambda x: x - 1 + coords[0],num_locs[0]), map(lambda y: y-1 + coords[1],num_locs[1]))
        for coords_2 in num_locs:
            adjs_2 = get_adjacent(temp_grid, coords_2)
            bomb_count = sum(adjs_2 == -3) + sum(adjs_2 == -4)
            mystery_count = sum(adjs_2 == -1)
        if bomb_count > adjs[4]:
            is_valid = False
            return is_valid
        if bomb_count < adjs[4] and mystery_count > 0:
            is_valid = False
            return is_valid
    return is_valid

In [ ]:
border_squares = get_border_mysteries()

In [ ]:
a = [1,2,3,4]
b = [4,5,6,4]
c = [7,8,9,4]
abc = [a,b,c]
zip(*abc)

In [ ]:
thing = np.array([1,2,3,4]) == 1

In [ ]:
thing == True

In [ ]:
hi

In [ ]:
temp_grid = grid
    mine_list = zip(np.ones(len(border_squares)), np.zeros(len(border_squares)))
    for coords in border_squares:
        thing = get_adjacent(temp_grid, coords)

In [ ]:
def backtest(lists, target_length):
    list_of_lists = []
    if len(lists) == 0:
        #print "I'm here"
        lis_one = np.array([1])
        print type(lis_one)
        lis_two = np.array([0])
        list_of_lists.append(lis_one)
        list_of_lists.append(lis_two)   
    else:
        #print "I'm down here"
        for arr in lists:
            lis_one = np.append(arr,0)
            lis_two = np.append(arr,1)
            #if check_valid(lis_one):
            list_of_lists.append(lis_one)
            #if check_valid(lis_two):
            list_of_lists.append(lis_two)   
    if len(lis_one) == target_length:
        return list_of_lists
    else:
        return backtest(list_of_lists, target_length)
            
        

In [ ]:
backtest([], 15)

In [ ]:
play_minesweeper()

In [ ]:
i = 1
j = 1
temp_grid = np.multiply(np.ones((grid.shape[0] + 2, grid.shape[1] + 2)),-2)
temp_grid[1:grid.shape[0] + 1, 1:grid.shape[1] + 1] = grid

In [ ]:
adj = np.concatenate((temp_grid[i+1][j-1:j+2],temp_grid[i][j-1:j+2],temp_grid[i-1][j-1:j+2]))
adj = np.delete(adj,4)

In [ ]:
adj

In [ ]:
temp_grid

In [ ]:
count_adjacent(a, (1,1), 8)


In [ ]:
np.multiply(temp_grid, -2)

In [ ]:
a =np.array([[3,4,5],[1,2,3],[7,8,9]])

In [ ]:
a

In [ ]:
a[(1,1),(0,0)]

In [ ]:
np.reshape(a,(1,9))

In [ ]:
b = np.

In [ ]:
b = np.ones((a.shape[0] + 2, a.shape[1] + 2))

b[1:a.shape[0] + 1, 1:a.shape[1] + 1] = a

In [ ]:
b

In [ ]:
len(np.where(a == 3))


In [ ]:
cv2.minMaxLoc(np.array([[1,0],[0,1]]))

In [ ]:
thing = np.concatenate((a[0][0:3], a[1][0:3], a[2][0:3]))

In [ ]:
np.delete(thing,4)

In [ ]:
np.reshape(thing, (3,3))


In [ ]:
a == 4

In [ ]:
a[(0,1)]

In [ ]:
def testy(c,d):
    global r
    r = c
    printytesty()
    

In [ ]:
def printytesty():
    print r

In [ ]:
testy(1,2)

In [ ]:
a

In [ ]:
a